In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"""
    IMAGE FORGERY ASSISTANT - INTERACTIVE DEMO
    +++++++++++++++++++++++++++++++++++++++++++

    FEATURES:
    1. Multi-agent system with specialized tools
    2. Hybrid architecture
    3. Interactive chat interface
    4. Real-tiem responses

    Developer: Praful Pawar
    
"""


In [2]:
# instal dependecies
!pip install -q google-generativeai ipywidgets 
print("Dependecies installed!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 6.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 46.0.3 which is incompatible.
pydrive2 1.21.3 requires pyOpenSSL<=24.2.1,>=19.1.0, but you have pyopenssl 25.3.0 which is incompatible.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2025.10.0 wh

In [3]:
#import required libraries

import os
import time
import json
from datetime import datetime
from typing import Dict, Any, Optional

#Google Generative Ai
from google import genai
from google.genai import types

#Kaggle Secrets
from kaggle_secrets import UserSecretsClient

#Widgets for interactive chat
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

print("Imports successful!")

Imports successful!


In [4]:
"Agent Configuration"
#Get API key from kaggle secrets
#important:add your google_api_key in kaggle secrets first
#go to : Add-ons -> secrets ->add secret
#Label: Google_API_Key
#Value:[your API key]

try:
    user_secrets = UserSecretsClient()
    GOOGLE_API_KEY = user_secrets.get_secret("Kaggle_Api_key")
    print("API Key loaded from secrets!")
except:
      print("Warning:Could not load API key from secrets")
      print("please add google api key to kaggle Secrets")

      GOOGLE_API_KEY= None


API Key loaded from secrets!


In [5]:
#Agent Configuration
AGENT_CONFIG ={
    "user_name" : "Praful",
    "competition": "Record.ai/LUC Scientific Image Forgery Detection",
    "current_model": "EfficientNet-B4 UNet++",
    "current_score": 0.303, 
    "target_score" :0.350,
    "version": "1.0.0"
}

#Display configuration
print("\n" + "="*70)
print("Agent Configuration")
print("="*70)
for key, value in AGENT_CONFIG.items():
    print(f". {key}:{value}")
print("="*70)


Agent Configuration
. user_name:Praful
. competition:Record.ai/LUC Scientific Image Forgery Detection
. current_model:EfficientNet-B4 UNet++
. current_score:0.303
. target_score:0.35
. version:1.0.0


In [6]:
"""
LLM Interface - Optimized for Kaggle
Uses Gemini API (cloud-based, no local server needed)
"""

import time
import google.generativeai as genai
from google.generativeai import types
from kaggle_secrets import UserSecretsClient


class KaggleLLM:
    """
    Simplified LLM for Kaggle environment
    Uses Google Gemini API
    """
    
    def __init__(self, api_key: str):
        if not api_key:
            raise ValueError("API key is required")

        # Configure Gemini
        genai.configure(api_key=api_key)
        
        self.client = genai
        self.model = "gemini-2.0-flash-exp"
        self.call_count = 0
        self.total_time = 0
        
        print(f"✅ LLM initialized with model: {self.model}")
    
    def call(
        self, 
        prompt: str, 
        max_tokens: int = 1000, 
        temperature: float = 0.3,
        system_prompt: str = None
    ) -> str:

        """
        Call Gemini API
        """

        try:
            # Combine system prompt if provided
            full_prompt = prompt
            if system_prompt:
                full_prompt = f"{system_prompt}\n\n{prompt}"
            
            # Track timing
            start_time = time.time()
            
            # Call API
            response = self.client.GenerativeModel(self.model).generate_content(
                full_prompt,
                generation_config=types.GenerationConfig(
                    max_output_tokens=max_tokens,
                    temperature=temperature
                )
            )
            
            # Update stats
            elapsed = time.time() - start_time
            self.call_count += 1
            self.total_time += elapsed
            
            return response.text
            
        except Exception as e:
            return f"❌ Error calling LLM: {str(e)}"
    
    def get_stats(self) -> dict:
        """Get LLM usage statistics"""
        
        avg_time = self.total_time / self.call_count if self.call_count > 0 else 0
        
        return {
            "total_calls": self.call_count,
            "total_time": round(self.total_time, 2),
            "avg_time": round(avg_time, 2)
        }


# ----------------------------------------------------------
# Load API Key from Kaggle Secrets
# ----------------------------------------------------------

user_secrets = UserSecretsClient()
GOOGLE_API_KEY = user_secrets.get_secret("Kaggle_Api_key")


# Initialize global LLM instance

if GOOGLE_API_KEY:
    llm = KaggleLLM(GOOGLE_API_KEY)
    print("✅ LLM ready to use!")
else:
    llm = None
    print("⚠️ LLM not initialized - API key missing")


✅ LLM initialized with model: gemini-2.0-flash-exp
✅ LLM ready to use!
